#PREGUNTA 7

7.      Programe el anterior problema en Python con

a.       El uso de DEAP

b.      Sin el uso de DEAP

In [2]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.2 MB/s eta 0:00:00


a. El uso de DEAP

In [6]:
import random
from deap import base, creator, tools, algorithms
import numpy as np

# 1. Definir el problema de maximización con la función objetivo f(x) = (x^(2x)) - 1
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# 2. Inicialización de individuos (población de números enteros aleatorios)
def init_individual():
    return [random.randint(0, 5)]  # Individuos entre 0 y 5

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 3. Definir la función objetivo f(x) = (x^(2x)) - 1
def eval_function(individual):
    x = individual[0]
    # Asegurarse de que la función no cause desbordamientos numéricos
    try:
        result = (x**(2 * x)) - 1
    except OverflowError:
        result = float('inf')  # Penaliza el valor si hay un desbordamiento
    return (result,)  # Devuelve una tupla con un único valor numérico

# 4. Registrar las operaciones genéticas
toolbox.register("evaluate", eval_function)
toolbox.register("mate", tools.cxBlend, alpha=0.5)    # Operador de cruce (blend crossover)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)  # Operador de mutación
toolbox.register("select", tools.selTournament, tournsize=3)  # Selección por torneo

# 5. Algoritmo genético con al menos 3 generaciones
def run_ga():
    # Inicializar la población
    population = toolbox.population(n=10)  # Población de 10 individuos

    # Configurar registros para guardar estadísticas
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Hall of Fame para guardar los mejores individuos
    hof = tools.HallOfFame(1)

    # Ejecutar el algoritmo genético por 3 generaciones
    print("Iniciando algoritmo genético...\n")
    result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, stats=stats, halloffame=hof, verbose=True)

    # Mostrar el mejor individuo
    print("\nMejor individuo encontrado:", hof[0])
    print("Fitness del mejor individuo:", hof[0].fitness.values[0])

    # Mostrar resumen estadístico
    print("\nResumen estadístico por generación:")
    for gen, log in enumerate(result[1]):
        print(f"Generación {gen + 1}: Promedio = {log['avg']}, Mejor = {log['max']}")

run_ga()


Iniciando algoritmo genético...

gen	nevals	avg    	std    	min	max  
0  	10    	13184.3	26176.2	0  	65535
1  	4     	2.47812e+09	7.43414e+09	15 	2.47805e+10
2  	7     	9.39101e+09	2.12388e+10	28192.5	6.91283e+10
3  	5     	1.8885e+10 	2.63759e+10	65535  	6.91283e+10
4  	6     	5.46216e+10	2.52507e+10	4.83979e+07	7.70317e+10
5  	5     	6.53737e+10	2.20653e+10	4.83979e+07	7.70317e+10

Mejor individuo encontrado: [6.627372060315224]
Fitness del mejor individuo: 77031696687.59828

Resumen estadístico por generación:
Generación 1: Promedio = 13184.3, Mejor = 65535.0
Generación 2: Promedio = 2478124468.5480804, Mejor = 24780540785.55872
Generación 3: Promedio = 9391007162.38507, Mejor = 69128295188.49797
Generación 4: Promedio = 18885026475.535004, Mejor = 69128295188.49797
Generación 5: Promedio = 54621568412.08329, Mejor = 77031696687.59828
Generación 6: Promedio = 65373722754.57212, Mejor = 77031696687.59828


b.      Sin el uso de DEAP

In [5]:
import random

# 1. Definir la función objetivo f(x) = (x^(2*x)) - 1
def eval_function(x):
    return (x**(2 * x)) - 1

# 2. Inicialización de la población
def init_population(size, lower_bound, upper_bound):
    population = [random.randint(lower_bound, upper_bound) for _ in range(size)]
    return population

# 3. Selección por torneo (escoge al mejor de un grupo aleatorio)
def tournament_selection(population, fitnesses, k=3):
    selected = random.sample(list(zip(population, fitnesses)), k)
    return max(selected, key=lambda ind_fit: ind_fit[1])[0]

# 4. Cruce (mezcla de genes de dos padres)
def crossover(parent1, parent2):
    alpha = 0.5
    return int(alpha * parent1 + (1 - alpha) * parent2)

# 5. Mutación (modificar el individuo ligeramente)
def mutate(individual, mutation_rate=0.1, lower_bound=0, upper_bound=10):
    if random.random() < mutation_rate:
        return random.randint(lower_bound, upper_bound)
    return individual

# 6. Algoritmo Genético
def genetic_algorithm(generations=3, population_size=10, lower_bound=0, upper_bound=5):
    # Inicializar población
    population = init_population(population_size, lower_bound, upper_bound)

    # Iterar a través de generaciones
    for gen in range(generations):
        print(f"\nGeneración {gen + 1}")

        # Evaluar aptitud de cada individuo
        fitnesses = [eval_function(ind) for ind in population]

        # Mostrar la población y sus aptitudes
        for i, (ind, fit) in enumerate(zip(population, fitnesses)):
            print(f"Individuo {i + 1}: {ind}, Fitness: {fit}")

        # Crear la nueva población
        new_population = []
        for _ in range(population_size // 2):
            # Seleccionar dos padres mediante torneo
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)

            # Cruzar padres para crear descendencia
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)

            # Mutar descendencia
            child1 = mutate(child1)
            child2 = mutate(child2)

            new_population.extend([child1, child2])

        # Reemplazar la población anterior con la nueva
        population = new_population

    # Evaluación final de la última generación
    final_fitnesses = [eval_function(ind) for ind in population]
    best_individual = population[final_fitnesses.index(max(final_fitnesses))]
    print(f"\nMejor individuo: {best_individual} con fitness: {max(final_fitnesses)}")

# Ejecutar el algoritmo genético
genetic_algorithm()



Generación 1
Individuo 1: 3, Fitness: 728
Individuo 2: 0, Fitness: 0
Individuo 3: 1, Fitness: 0
Individuo 4: 5, Fitness: 9765624
Individuo 5: 2, Fitness: 15
Individuo 6: 1, Fitness: 0
Individuo 7: 5, Fitness: 9765624
Individuo 8: 5, Fitness: 9765624
Individuo 9: 2, Fitness: 15
Individuo 10: 2, Fitness: 15

Generación 2
Individuo 1: 3, Fitness: 728
Individuo 2: 3, Fitness: 728
Individuo 3: 3, Fitness: 728
Individuo 4: 3, Fitness: 728
Individuo 5: 7, Fitness: 678223072848
Individuo 6: 0, Fitness: 0
Individuo 7: 3, Fitness: 728
Individuo 8: 7, Fitness: 678223072848
Individuo 9: 2, Fitness: 15
Individuo 10: 1, Fitness: 0

Generación 3
Individuo 1: 5, Fitness: 9765624
Individuo 2: 5, Fitness: 9765624
Individuo 3: 5, Fitness: 9765624
Individuo 4: 5, Fitness: 9765624
Individuo 5: 3, Fitness: 728
Individuo 6: 3, Fitness: 728
Individuo 7: 5, Fitness: 9765624
Individuo 8: 5, Fitness: 9765624
Individuo 9: 5, Fitness: 9765624
Individuo 10: 5, Fitness: 9765624

Mejor individuo: 5 con fitness: 9765

In [13]:
import random
import numpy as np

# Definir la función objetivo f(x) = (x^(2*x)) - 1
def evalFunction(x):
    if x > 0:
        return (x ** (2 * x)) - 1
    else:
        return float('inf')  # Penalización para valores no válidos

# Generar una población inicial
def create_population(size, min_val, max_val):
    return [random.randint(min_val, max_val) for _ in range(size)]

# Selección por torneo
def tournament_selection(population, fitnesses, k=3):
    selected = []
    for _ in range(len(population)):
        aspirants = random.sample(list(zip(population, fitnesses)), k)
        selected.append(min(aspirants, key=lambda ind: ind[1])[0])
    return selected

# Cruce de dos puntos
def crossover(parent1, parent2):
    if random.random() < 0.5:
        return parent1, parent2  # Sin cruce
    return (parent1 + parent2) // 2, (parent1 + parent2) // 2

# Mutación gaussiana
def mutate(individual, mutation_rate=0.2):
    if random.random() < mutation_rate:
        return individual + int(random.gauss(0, 1))
    return individual

# Ejecutar el algoritmo genético
def genetic_algorithm(pop_size=10, generations=3, min_val=1, max_val=10):
    population = create_population(pop_size, min_val, max_val)

    for gen in range(generations):
        # Evaluar la aptitud de la población
        fitnesses = [evalFunction(ind) for ind in population]

        print(f"Generación {gen}, Mejor Fitness: {min(fitnesses)}")

        # Selección
        selected = tournament_selection(population, fitnesses)

        # Cruce y mutación
        offspring = []
        for i in range(0, len(selected), 2):
            if i + 1 < len(selected):
                child1, child2 = crossover(selected[i], selected[i + 1])
                offspring.append(mutate(child1))
                offspring.append(mutate(child2))

        population = offspring

    # Mejor individuo
    final_fitnesses = [evalFunction(ind) for ind in population]
    best_individual = population[np.argmin(final_fitnesses)]
    print(f"Mejor individuo final: {best_individual}, Fitness: {min(final_fitnesses)}")

if __name__ == "__main__":
    genetic_algorithm()


Generación 0, Mejor Fitness: 0
Generación 1, Mejor Fitness: 0
Generación 2, Mejor Fitness: 0
Mejor individuo final: 1, Fitness: 0
